In [20]:
import os
import random
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pylab as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import mnist


In [ ]:
df = pd.read_csv("binary_data.csv")

df['id_code'] = df['id_code'].astype(str) + '.png'

X = df["id_code"]
y = df["diagnosis"]


df['filepath'] = df.apply(lambda row: os.path.join("ADNI_IMAGES",
                                                   'AD' if row['diagnosis'] == 4 else 'CN',
                                                   row['id_code']), axis=1)

train_df, temp_df = train_test_split(df, test_size=0.4, stratify=df['diagnosis'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['diagnosis'], random_state=42)

train_df['diagnosis'] = train_df['diagnosis'].map({0: 'CN', 4: 'AD'})
val_df['diagnosis'] = val_df['diagnosis'].map({0: 'CN', 4: 'AD'})
test_df['diagnosis'] = test_df['diagnosis'].map({0: 'CN', 4: 'AD'})

datagen = ImageDataGenerator(rescale=1./255)


train_data = datagen.flow_from_dataframe(train_df, x_col='filepath', y_col='diagnosis',
                                         target_size=(224, 224), batch_size=32,
                                         class_mode='binary', shuffle=True)

val_data = datagen.flow_from_dataframe(val_df, x_col='filepath', y_col='diagnosis',
                                       target_size=(224, 224), batch_size=32,
                                       class_mode='binary', shuffle=False)

test_data = datagen.flow_from_dataframe(test_df, x_col='filepath', y_col='diagnosis',
                                        target_size=(224, 224), batch_size=32,
                                        class_mode='binary', shuffle=False)


Found 4891 validated image filenames belonging to 2 classes.
Found 1630 validated image filenames belonging to 2 classes.
Found 1631 validated image filenames belonging to 2 classes.
True


,id_code,diagnosis,filepath
6289,AD-3981.png,AD,ADNI_IMAGES/AD/AD-3981.png
5378,AD-2647.png,AD,ADNI_IMAGES/AD/AD-2647.png
1170,CN-3105.png,CN,ADNI_IMAGES/CN/CN-3105.png
5384,AD-0726.png,AD,ADNI_IMAGES/AD/AD-0726.png
2234,CN-1313.png,CN,ADNI_IMAGES/CN/CN-1313.png
...,...,...,...
294,CN-3465.png,CN,ADNI_IMAGES/CN/CN-3465.png
5417,AD-3284.png,AD,ADNI_IMAGES/AD/AD-3284.png
6461,AD-2752.png,AD,ADNI_IMAGES/AD/AD-2752.png
597,CN-4078.png,CN,ADNI_IMAGES/CN/CN-4078.png


In [37]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(1, activation='sigmoid')  # <--- binary output (Alzheimer's vs Normal)
])


model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(
    train_data,
    validation_data=val_data,
    epochs=20
)


Epoch 1/20


/Users/jillian/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/jillian/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


153/153 ━━━━━━━━━━━━━━━━━━━━ 58s 374ms/step - accuracy: 0.4985 - loss: 0.8106 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/20
153/153 ━━━━━━━━━━━━━━━━━━━━ 58s 379ms/step - accuracy: 0.4992 - loss: 0.6932 - val_accuracy: 0.4865 - val_loss: 0.6932
Epoch 3/20
153/153 ━━━━━━━━━━━━━━━━━━━━ 59s 384ms/step - accuracy: 0.4986 - loss: 0.6931 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 4/20
153/153 ━━━━━━━━━━━━━━━━━━━━ 60s 390ms/step - accuracy: 0.5029 - loss: 0.6932 - val_accuracy: 0.4368 - val_loss: 0.6932
Epoch 5/20
  9/153 ━━━━━━━━━━━━━━━━━━━━ 54s 378ms/step - accuracy: 0.5213 - loss: 0.6931

KeyboardInterrupt: 